In [1]:
import numpy as np
import pandas as pd
import os
import time
import sys

#bla bla

import sklearn
import psutil
import pickle

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from copy import deepcopy as dc

### Import Dati

In [2]:
with open('..\\dati\\step_0_dati_out.pickle', 'rb') as f:
    df = pickle.load(file=f)
y = df["survived"]
df = df.drop(columns = "survived")
df.head(3)

,age,fare,embarked,sex,pclass,body,cabin
0,29.0000,211.3375,S,female,1.0,NaN,B5
1,0.9167,151.5500,S,male,1.0,NaN,C22 C26
2,2.0000,151.5500,S,female,1.0,NaN,C22 C26


In [3]:
step_1 = {}

### Imputation NA

In [4]:
for col in df.columns:
    if df[col].isna().mean()>0.65:
        df.drop(col,axis=1, inplace = True)
        print(f"{col} dropped")

body dropped
cabin dropped


In [5]:
numeric_features = ["age", "fare"]
numeric_imputation = ColumnTransformer(
    transformers=[("numeric imp",SimpleImputer(strategy="median"), numeric_features)])

fit_num_NA = numeric_imputation.fit(df)
fitted_num_NA = dc(fit_num_NA)
step_1["step_1_imputation_num"] = {"obj":fitted_num_NA,
                                          "fitted_on":numeric_features}
df.loc[::,numeric_features] = fit_num_NA.transform(df)
df.shape

(1309, 5)

In [6]:
categorical_features = ["embarked", "sex", "pclass"]
categorical_imputation = ColumnTransformer(
    transformers=[("cat imp",SimpleImputer(strategy="most_frequent"), categorical_features)])
fit_cat_NA = categorical_imputation.fit(df)
fitted_cat_NA = dc(fit_cat_NA)
step_1["step_1_imputation_cat"] = {"obj":fitted_cat_NA,
                                          "fitted_on":categorical_features}
df.loc[::,categorical_features] = fit_cat_NA.transform(df)
df.shape

(1309, 5)

In [7]:
assert df.isnull().sum().sum() <=0

In [8]:
df = pd.concat([df,y], axis=1)

### Salvataggio Pickle

In [9]:
save_path = os.getenv('PATH_NAME',
        'C:\\Users\\AndreaNicoletta\\Desktop\\Elyra\\pipelines\\titanic\\dati\\')

#salvataggio dict con oggetti fittati
with open(save_path+'step_1_dict_out.pickle', 'wb') as handle:
    pickle.dump(step_1, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
#salvataggio dati su cui è stato chiamato il transform
with open(save_path+'step_1_dati_out.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)